# Simulation of Projectile Motion in 2D - no air resistance.

In [1]:
import numpy as np

# Define the initial conditions of the motion
v0 = 6.96
theta_degrees = 0.0
y0 = 1.1


theta = theta_degrees*np.pi/180.0
v0y = v0*np.sin(theta)
x0 = 0.0
v0x = v0*np.cos(theta)
ay = -9.80524
ax = 0.0

In [2]:
# Define the time interval, dt, for the simulation
dt = 0.001

y = []
x = []
t = []
vy = []
vx = []

y.append(y0)
x.append(x0)
vy.append(v0y)
vx.append(v0x)
t.append(0.0)

y_current = y[0]
x_current = x[0]
vy_current = vy[0]
vx_current = vx[0]
t_current = t[0]

# Theory:
# v = v0 + a_yt
# y = y0 + v0y*t + 1/2*a_y*t**2
# x = x0 + v0x*t + 1/2*a_x*t**2

n=1
while (y_current > 0):
    y_current += vy_current*dt
    x_current += vx_current*dt
    vy_current += ay*dt
    vx_current += ax*dt
    t_current += dt
    y.append(y_current)
    x.append(x_current)
    vy.append(vy_current)
    vx.append(vx_current)
    t.append(t_current)
    n = n + 1


In [3]:
import matplotlib.pyplot as plt

# Generate the figure and the axes 
fig, axs = plt.subplots(nrows=2, ncols=2) 

# On the first axis, plot the y value 
axs[0][0].plot(t,y) 
axs[0][0].set_ylabel('y') 

# On the second axis, plot the vy value 
axs[1][0].plot(t,vy) 
axs[1][0].set_ylabel('vy')

# On the third axis, plot the x value 
axs[0][1].plot(t,x) 
axs[0][1].set_ylabel('x') 

# On the fourth axis, plot the vx value 
axs[1][1].plot(t,vx) 
axs[1][1].set_ylabel('vx')

plt.show()

<Figure size 640x480 with 4 Axes>

In [4]:
from matplotlib import animation
from matplotlib.patches import Ellipse
from IPython.display import HTML
import numpy as np

for i in range(30,120):
    FPS = i
    framesNum = int(FPS*t[-1])+1
    if (n%framesNum==0):
        break
        
print ("n, FPS, framesNum = ", n,FPS,framesNum)
plt.style.use('default')

x = np.array(x)
y = np.array(y)
xm = x.min()
xp = x.max()
ym = y.min()
yp = y.max()*1.1
print ("X_min, X_max = ",xm,xp)
print ("Y_min, Y_max = ",ym,yp/1.1)

# Set up the figure
fig = plt.figure(figsize=(10, 5), dpi=60)
ax = plt.axes(xlim=(xm, xp), ylim=(ym, yp))

#ax.set_aspect('equal')
ax.axis('on')

# Define the different elements in the animation
ball = Ellipse((x[0], y[0]),0.0185*xp,0.0369*yp,
             facecolor = 'blue',
             fill=True)
ax.add_patch(ball)

# Animation function. This is called sequentially.
#print(n,framesNum)
def animate(j):
    i = (j+1)*int(n/framesNum)-1
    ball.center = (x[i], y[i])

# Create animation
anim = animation.FuncAnimation(fig, animate, frames=framesNum, interval=1200/FPS, repeat=False)
plt.close(anim._fig)

# Display the animation
HTML(anim.to_html5_video())

n, FPS, framesNum =  476 34 17
X_min, X_max =  0.0 3.3059999999999823
Y_min, Y_max =  -0.0038248929999978813 1.1
